# Unsupervised Classification of AOP Surface Directional Reflectance data in GEE

Authors: Bridget Hass, John Musinsky, Qiusheng Wu

Modified from Qiusheng Wu's [Machine Learning with Earth Engine - Unsupervised Classification](https://github.com/giswqs/geemap/blob/master/examples/notebooks/31_unsupervised_classification.ipynb)

Source: https://developers.google.com/earth-engine/clustering

The `ee.Clusterer` package handles unsupervised classification (or clustering) in Earth Engine. More details about each Clusterer are available in the reference docs in the Code Editor.

In this tutorial, we will use a GEE clusterer to carry out unsupervised classification of SRER, using data from 2017 (collected in good weather conditions "green"), and compare it to data from 2018 (mixed weather conditions).

In [1]:
import ee, geemap

In [4]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWjSAxDZzdlX-YBptmw7CpD7I8izr-OqdIpE9fCIXuF9p-Gvk-k8fbI



Successfully saved authorization token.


In [5]:
ee.Initialize()

## Create an interactive map

In [6]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Add SRER 2017 Surface Directional Reflectance

In [7]:
# Specify center location of SRER
geo = ee.Geometry.Point([-110.83549, 31.91068])

# Read in Surface Directional Reflectance (SDR) Images and filter by date and bounds 
# SRER_SDR2017 = ee.ImageCollection("projects/neon/DP3-30006-001_SDR").filterDate('2017-01-01', '2017-12-31').filterBounds(geo).first();
SRER_SDR2021 = ee.Image("projects/neon/D14_SRER/L3/DP3-30006-001_D14_SRER_SDR_2021")

# Mask layers to only show values > 0 (this hides the no data values of -9999) 
# SRER_SDR2017mask = SRER_SDR2017.updateMask(SRER_SDR2017.gte(0.0000))
SRER_SDR2021mask = SRER_SDR2021.updateMask(SRER_SDR2021.gte(0.0000))

# Set the visualization parameters so contrast is maximized, and set display to show RGB bands 
visParams = {'min':2,'max':20,'gamma':0.9,'bands':['band053','band035','band019']};

Map.addLayer(SRER_SDR2021mask, visParams, 'SRER 2021');

Map.setCenter(-110.83549, 31.91068, 10);

In [14]:
%pprint

Pretty printing has been turned OFF


## Remove water vapor absorption bands!

In [16]:
bands = SRER_SDR2021.get('system:band_names').getInfo()
good_bands = [b for b in bands if b not in ['band' + str(a) for a in range(190,212)] and b not in ['band' + str(a) for a in range(280,314)]]

In [20]:
len(good_bands)

370

In [19]:
SRER2021_good_bands = SRER_SDR2021.select(good_bands)
bands = SRER2021_good_bands.bandNames()
bands.getInfo()

['band001', 'band002', 'band003', 'band004', 'band005', 'band006', 'band007', 'band008', 'band009', 'band010', 'band011', 'band012', 'band013', 'band014', 'band015', 'band016', 'band017', 'band018', 'band019', 'band020', 'band021', 'band022', 'band023', 'band024', 'band025', 'band026', 'band027', 'band028', 'band029', 'band030', 'band031', 'band032', 'band033', 'band034', 'band035', 'band036', 'band037', 'band038', 'band039', 'band040', 'band041', 'band042', 'band043', 'band044', 'band045', 'band046', 'band047', 'band048', 'band049', 'band050', 'band051', 'band052', 'band053', 'band054', 'band055', 'band056', 'band057', 'band058', 'band059', 'band060', 'band061', 'band062', 'band063', 'band064', 'band065', 'band066', 'band067', 'band068', 'band069', 'band070', 'band071', 'band072', 'band073', 'band074', 'band075', 'band076', 'band077', 'band078', 'band079', 'band080', 'band081', 'band082', 'band083', 'band084', 'band085', 'band086', 'band087', 'band088', 'band089', 'band090', 'band091'

## Look at the image properties

props = geemap.image_props(SRER_SDR2021)
props.getInfo()

In [10]:
props.get('IMAGE_DATE').getInfo()

'2021-08-23'

## Make training dataset
There are several ways you can create a region for generating the training dataset.

Draw a shape (e.g., rectangle) on the map and the use region = Map.user_roi
Define a geometry, such as region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
Create a buffer zone around a point, such as region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)
If you don't define a region, it will use the image footprint by default

In [11]:
# Make the training dataset.
training = SRER_SDR2021.sample(**{
#     'region': region,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=426812.0, center=[31.798224014917242, -110.7608985900879], controls=(WidgetControl(options=['positi…

## Train the clusterer

In [12]:
# Instantiate the clusterer and train it.
n_clusters = 4
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

## Classify the image

In [13]:
# Cluster the input using the trained clusterer.
result = SRER_SDR2021.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=426812.0, center=[31.798224014917242, -110.7608985900879], controls=(WidgetControl(options=['positi…

## Label the clusters

In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072']

# Reclassify the map
result = result.remap([0, 1, 2, 3], [1, 2, 3, 4])

Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

In [ ]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')